# Test Pipeline 4:

In [1]:
import json
import re
from pathlib import Path
from docx import Document
from tqdm import tqdm
import nltk
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForQuestionAnswering,
    pipeline
)

nltk.download("punkt")

# Load Models
qg_tokenizer = T5Tokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
qg_model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap").to("cuda")

qa_tokenizer = AutoTokenizer.from_pretrained("/home/ec2-user/qa_roberta_telecom")
qa_model = AutoModelForQuestionAnswering.from_pretrained("/home/ec2-user/qa_roberta_telecom").to("cuda")
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0)

# Text Cleaning
def clean_technical_text(raw_text: str) -> str:
    cleaned_lines = []
    last_heading = None
    boilerplate_keywords = [
        "confidential", "etsi", "3gpp", "table of contents",
        "appendix", "index", "page", "internal use only"
    ]

    for line in raw_text.split("\n"):
        line = line.strip()
        if not line or (len(line.split()) < 4 and ":" not in line):
            continue
        if any(kw in line.lower() for kw in boilerplate_keywords):
            continue
        if re.match(r"^(figure|fig\.|table)\s*\d+", line, re.IGNORECASE):
            continue
        if re.match(r"^\d+(\.\d+){0,4}\s+[A-Z]", line):
            if len(line.split()) < 3:
                continue
            if any(kw in line.lower() for kw in boilerplate_keywords):
                continue
            last_heading = line
            continue
        if line.isupper() and len(line.split()) < 10:
            continue

        # Remove reference brackets and long citations
        line = re.sub(r"\[\d+\]", "", line)
        line = re.sub(r"\(ETSI.*?\)", "", line)

        if last_heading:
            line = f"{last_heading}. {line}"
            last_heading = None
        cleaned_lines.append(line)

    return " ".join(cleaned_lines)

# Chunking
def split_with_overlap(text, max_words=200, overlap=20):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + max_words]
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks

# Sentence Selector

def select_highlight_sentence(chunk, strategy="length"):
    sentences = [s.strip() for s in re.split(r'(?<=[.?!])\s+(?=[A-Z])', chunk.strip()) if len(s.strip().split()) > 3]
    if not sentences:
        return None
    if strategy == "length":
        return max(sentences, key=len)
    elif strategy == "tfidf":
        try:
            tfidf = TfidfVectorizer().fit(sentences)
            X = tfidf.transform(sentences)
            scores = X.mean(axis=1).A1
            return sentences[scores.argmax()]
        except Exception:
            return max(sentences, key=len)
    else:
        raise ValueError("Invalid strategy for sentence selection")

# Question Generation
def batch_generate_questions(inputs, max_length=64):
    tokenized = qg_tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output_ids = qg_model.generate(**tokenized, max_length=max_length)
    return [qg_tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]

# QA Generation
def generate_qa_pairs_from_paragraphs(paragraphs: list[str], source: str, batch_size=16, strategy="length"):
    qa_pairs = []
    counters = {"total_paragraphs": len(paragraphs), "qa_generated": 0, "qa_fallback": 0, "skipped_filters": 0, "errors": 0}

    for i in tqdm(range(0, len(paragraphs), batch_size), desc="Generating QA pairs"):
        batch_paras = paragraphs[i:i+batch_size]
        q_inputs, batch_indices = [], []

        for j, para in enumerate(batch_paras):
            idx = i + j
            if len(para.split()) < 3:
                continue
            if re.search(r'\d+\.\d+.*?:?', para) and len(para) < 400:
                continue
            if (sum(c.isdigit() for c in para) / max(len(para), 1)) > 0.6:
                continue

            sent = select_highlight_sentence(para, strategy=strategy)
            if not sent:
                continue
            highlight = para.replace(sent, f"<hl> {sent} <hl>")
            q_inputs.append(f"highlight: {highlight}")
            batch_indices.append((idx, para, sent, para.split(".")))

        if not q_inputs:
            continue

        try:
            tokenized = qg_tokenizer(q_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
            with torch.no_grad():
                output_ids = qg_model.generate(**tokenized, max_length=64, num_beams=2)
            questions = [{"generated_text": qg_tokenizer.decode(ids, skip_special_tokens=True)} for ids in output_ids]
        except Exception as e:
            print(f"[ERROR] QG batch failed: {e}")
            counters["errors"] += len(q_inputs)
            continue

        qa_inputs = [
            {"question": q["generated_text"], "context": para}
            for (_, para, _, _), q in zip(batch_indices, questions)
            if not q["generated_text"].lower().startswith("what is the highlight")
        ]

        if not qa_inputs:
            continue

        try:
            questions_list = [qa["question"] for qa in qa_inputs]
            contexts_list = [qa["context"] for qa in qa_inputs]
            answers = qa_pipeline(question=questions_list, context=contexts_list)
            if isinstance(answers, dict):
                answers = [answers]
        except Exception as e:
            print(f"[ERROR] QA batch failed: {e}")
            counters["errors"] += len(qa_inputs)
            continue

        for (idx, para, sent, sentences), question, result in zip(batch_indices, questions, answers):
            question_text = question["generated_text"]
            if question_text.lower().startswith("what is the highlight"):
                continue
            answer, score = result["answer"].strip(), result["score"]
            if score < 0.15 or len(answer) < 4 or not any(c.isalnum() for c in answer):
                counters["skipped_filters"] += 1
                continue
            if answer.lower() in ["yes", "no", "maybe"]:
                counters["skipped_filters"] += 1
                continue
            if not any(answer.lower() in s.lower() for s in sentences):
                counters["skipped_filters"] += 1
                continue

            qa_pairs.append({
                "instruction": "Extract the correct answer span from the telecom document context.",
                "input": f"### Task: extractive_qa\n### Context:\n{para}\n\n### Question:\n{question_text}\n\n### Answer:",
                "output": answer,
                "source_doc": source,
                "chunk_id": idx,
                "confidence": round(score, 3)
            })
            counters["qa_generated"] += 1

    print(f"\U0001F50D Total Paragraphs: {counters['total_paragraphs']}")
    print(f" QA Pairs: {counters['qa_generated']}")
    print(f" Skipped: {counters['skipped_filters']} |  Errors: {counters['errors']}")
    return qa_pairs

# DOCX Processing
def process_docx_file(input_docx_path, output_jsonl_path, strategy="length"):
    """
    Load DOCX file, clean text, chunk with overlap, generate QA pairs, and save output.
    Uses tighter chunking: 150 words with 50-word overlap.
    """
    input_path = Path(input_docx_path)
    doc = Document(input_path)

    # Combine and clean text
    raw_text = "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])
    cleaned = clean_technical_text(raw_text)

    # Apply tighter chunking for better QA coverage
    paragraphs = split_with_overlap(cleaned, max_words=150, overlap=50)

    # Generate QA pairs from processed paragraphs
    qa_pairs = generate_qa_pairs_from_paragraphs(paragraphs, source=input_path.name, strategy=strategy)

    # Save as JSONL
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for pair in qa_pairs:
            f.write(json.dumps(pair, ensure_ascii=False) + '\n')

    print(f"✅ Saved {len(qa_pairs)} QA pairs to {output_jsonl_path}")

# Filtering Good QA Pairs
def filter_good_qa_pairs(input_jsonl_path, output_jsonl_path):
    qa_pairs = []
    with open(input_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            qa_pairs.append(json.loads(line))

    qa_df = pd.DataFrame(qa_pairs)
    qa_df["question"] = qa_df["input"].str.extract(r"### Question:\n(.+?)\n\n### Answer:")
    qa_df["context"] = qa_df["input"].str.extract(r"### Context:\n(.+?)\n\n### Question:", flags=re.DOTALL)

    qa_df["low_confidence_flag"] = qa_df["confidence"].astype(float) < 0.4
    qa_df["mismatch_flag"] = ~qa_df.apply(lambda row: row["output"].lower() in row["context"].lower(), axis=1)
    qa_df["vague_question_flag"] = qa_df["question"].str.lower().str.startswith("what is the highlight")

    qa_df["quality"] = "good"
    qa_df.loc[qa_df["low_confidence_flag"] | qa_df["mismatch_flag"] | qa_df["vague_question_flag"], "quality"] = "poor"

    good_qa_df = qa_df[qa_df["quality"] == "good"]
    with open(output_jsonl_path, "w", encoding="utf-8") as fout:
        for record in good_qa_df.to_dict(orient="records"):
            json.dump(record, fout, ensure_ascii=False)
            fout.write("\n")

    print(f" Saved {len(good_qa_df)} good QA pairs to {output_jsonl_path}")

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0


# To run Pipeline on Single file for testing:

In [ ]:
process_docx_file(
    input_docx_path="/mnt/data/Datasets/38104-j00.docx",
    output_jsonl_path="/mnt/data/qa_output.jsonl",
    strategy="tfidf"
)

In [ ]:
filter_good_qa_pairs(
    input_jsonl_path="/mnt/data/qa_output.jsonl",
    output_jsonl_path="/mnt/data/qa_output_filtered_good.jsonl"
)

# To run the Pipeline on whole 3GPP documents from TSpec-LLM:

In [3]:
from pathlib import Path
import shutil
import pandas as pd
import json

# Paths
base_path = Path("/mnt/data/TSpec-LLM/3GPP-clean")
temp_dir = Path("/mnt/data/temp_qa_outputs")
final_output = Path("/mnt/data/final_3gpp_qa_filtered.jsonl")
stats_output = Path("/mnt/data/qa_stats_per_file.csv")

# Reset
shutil.rmtree(temp_dir, ignore_errors=True)
temp_dir.mkdir(parents=True, exist_ok=True)
final_output.unlink(missing_ok=True)

# Limit files for testing
docx_files = sorted(base_path.rglob("*.docx"))

stats = []

for docx_path in docx_files:
    file_id = docx_path.stem.replace(" ", "_")
    raw_path = temp_dir / f"{file_id}_raw.jsonl"
    filtered_path = temp_dir / f"{file_id}_filtered.jsonl"

    try:
        process_docx_file(docx_path, raw_path, strategy="tfidf")
        filter_good_qa_pairs(raw_path, filtered_path)

        with open(filtered_path, "r", encoding="utf-8") as f:
            count = sum(1 for _ in f)

        stats.append({"file": docx_path.name, "qa_count": count, "status": "success"})

        with open(filtered_path, "r", encoding="utf-8") as fin, open(final_output, "a", encoding="utf-8") as fout:
            fout.writelines(fin.readlines())

    except Exception as e:
        stats.append({"file": docx_path.name, "qa_count": 0, "status": f"error: {str(e)}"})

# Save stats
stats_df = pd.DataFrame(stats)
stats_df.to_csv(stats_output, index=False)
stats_df.head()

Generating QA pairs: 100%|████████████████████████| 1/1 [00:00<00:00,  3.03it/s]


🔍 Total Paragraphs: 2
✅ QA Pairs: 2
🚫 Skipped: 0 | ❌ Errors: 0
✅ Saved 2 QA pairs to /mnt/data/temp_qa_outputs/21101-a40_raw.jsonl
✅ Saved 1 good QA pairs to /mnt/data/temp_qa_outputs/21101-a40_filtered.jsonl


Generating QA pairs: 100%|████████████████████████| 1/1 [00:00<00:00,  2.84it/s]


🔍 Total Paragraphs: 1
✅ QA Pairs: 0
🚫 Skipped: 1 | ❌ Errors: 0
✅ Saved 0 QA pairs to /mnt/data/temp_qa_outputs/21101_CR0074_(Rel-10)_raw.jsonl


Generating QA pairs: 100%|████████████████████████| 2/2 [00:01<00:00,  1.12it/s]


...

🔍 Total Paragraphs: 17
✅ QA Pairs: 8
🚫 Skipped: 9 | ❌ Errors: 0
✅ Saved 8 QA pairs to /mnt/data/temp_qa_outputs/26073-a00_raw.jsonl
✅ Saved 6 good QA pairs to /mnt/data/temp_qa_outputs/26073-a00_filtered.jsonl


Generating QA pairs: 100%|████████████████████████| 3/3 [00:02<00:00,  1.09it/s]


🔍 Total Paragraphs: 36
✅ QA Pairs: 7
🚫 Skipped: 29 | ❌ Errors: 0
✅ Saved 7 QA pairs to /mnt/data/temp_qa_outputs/26074-a00_raw.jsonl
✅ Saved 5 good QA pairs to /mnt/data/temp_qa_outputs/26074-a00_filtered.jsonl


Generating QA pairs:   0%|                                | 0/8 [00:01<?, ?it/s]


KeyboardInterrupt: 

# Final Main Pipeline wth Abbrevation concept:

In [1]:
import json
import re
from pathlib import Path
from docx import Document
from tqdm import tqdm
import nltk
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForQuestionAnswering,
    pipeline
)

nltk.download("punkt")

# Load Models
qg_tokenizer = T5Tokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
qg_model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap").to("cuda")

qa_tokenizer = AutoTokenizer.from_pretrained("/home/ec2-user/qa_roberta_telecom")
qa_model = AutoModelForQuestionAnswering.from_pretrained("/home/ec2-user/qa_roberta_telecom").to("cuda")
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0)

# Load abbreviation map
with open("abbreviation_master_map.json", "r", encoding="utf-8") as f:
    abbreviation_map = json.load(f)
    
def expand_abbreviations(text, abbreviation_map):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in abbreviation_map.keys()) + r')\b')
    return pattern.sub(lambda m: f"{abbreviation_map[m.group()]} ({m.group()})", text)

# Text Cleaning
def clean_technical_text(raw_text: str) -> str:
    cleaned_lines = []
    last_heading = None
    boilerplate_keywords = [
        "confidential", "etsi", "3gpp", "table of contents",
        "appendix", "index", "page", "internal use only"
    ]

    for line in raw_text.split("\n"):
        line = line.strip()
        if not line or (len(line.split()) < 4 and ":" not in line):
            continue
        if any(kw in line.lower() for kw in boilerplate_keywords):
            continue
        if re.match(r"^(figure|fig\.|table)\s*\d+", line, re.IGNORECASE):
            continue
        if re.match(r"^\d+(\.\d+){0,4}\s+[A-Z]", line):
            if len(line.split()) < 3:
                continue
            if any(kw in line.lower() for kw in boilerplate_keywords):
                continue
            last_heading = line
            continue
        if line.isupper() and len(line.split()) < 10:
            continue

        # Remove reference brackets and long citations
        line = re.sub(r"\[\d+\]", "", line)
        line = re.sub(r"\(ETSI.*?\)", "", line)

        if last_heading:
            line = f"{last_heading}. {line}"
            last_heading = None
        cleaned_lines.append(line)

    return " ".join(cleaned_lines)

# Chunking
def split_with_overlap(text, max_words=200, overlap=20):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + max_words]
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks

# Sentence Selector

# def select_highlight_sentence(chunk, strategy="length"):
#     sentences = [s.strip() for s in re.split(r'(?<=[.?!])\s+(?=[A-Z])', chunk.strip()) if len(s.strip().split()) > 3]
#     if not sentences:
#         return None
#     if strategy == "length":
#         return max(sentences, key=len)
#     elif strategy == "tfidf":
#         try:
#             tfidf = TfidfVectorizer().fit(sentences)
#             X = tfidf.transform(sentences)
#             scores = X.mean(axis=1).A1
#             return sentences[scores.argmax()]
#         except Exception:
#             return max(sentences, key=len)
#     else:
#         raise ValueError("Invalid strategy for sentence selection")

def select_highlight_sentence(chunk, strategy="length"):
    # Split into sentences
    sentences = [s.strip() for s in re.split(r'(?<=[.?!])\s+(?=[A-Z])', chunk.strip()) if len(s.strip().split()) > 3]

    if not sentences:
        return None

    if strategy == "length":
        # Original logic: longest sentence
        return max(sentences, key=len)

    elif strategy == "tfidf":
        # Lightweight approximation pick sentence with highest unique token ratio
        def score(s):
            words = s.lower().split()
            return len(set(words)) / max(len(words), 1)

        return max(sentences, key=score)

    else:
        raise ValueError("Invalid strategy for sentence selection")

# Question Generation
def batch_generate_questions(inputs, max_length=64):
    tokenized = qg_tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output_ids = qg_model.generate(**tokenized, max_length=max_length)
    return [qg_tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]

# QA Generation
def generate_qa_pairs_from_paragraphs(paragraphs: list[str], source: str, batch_size=64, strategy="length"):
    qa_pairs = []
    counters = {"total_paragraphs": len(paragraphs), "qa_generated": 0, "qa_fallback": 0, "skipped_filters": 0, "errors": 0}

    for i in tqdm(range(0, len(paragraphs), batch_size), desc="Generating QA pairs"):
        batch_paras = paragraphs[i:i+batch_size]
        q_inputs, batch_indices = [], []

        for j, para in enumerate(batch_paras):
            idx = i + j
            if len(para.split()) < 3:
                continue
            if re.search(r'\d+\.\d+.*?:?', para) and len(para) < 400:
                continue
            if (sum(c.isdigit() for c in para) / max(len(para), 1)) > 0.6:
                continue

            sent = select_highlight_sentence(para, strategy=strategy)
            if not sent:
                continue
            highlight = para.replace(sent, f"<hl> {sent} <hl>")
            q_inputs.append(f"highlight: {highlight}")
            batch_indices.append((idx, para, sent, para.split(".")))

        if not q_inputs:
            continue

        try:
            tokenized = qg_tokenizer(q_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
            with torch.no_grad():
                output_ids = qg_model.generate(**tokenized, max_length=64, num_beams=2)
            questions = [{"generated_text": qg_tokenizer.decode(ids, skip_special_tokens=True)} for ids in output_ids]
        except Exception as e:
            print(f"[ERROR] QG batch failed: {e}")
            counters["errors"] += len(q_inputs)
            continue

        qa_inputs = [
            {"question": q["generated_text"], "context": para}
            for (_, para, _, _), q in zip(batch_indices, questions)
            if not q["generated_text"].lower().startswith("what is the highlight")
        ]

        if not qa_inputs:
            continue

        try:
            questions_list = [qa["question"] for qa in qa_inputs]
            contexts_list = [qa["context"] for qa in qa_inputs]
            answers = qa_pipeline(question=questions_list, context=contexts_list, batch_size=64)
            if isinstance(answers, dict):
                answers = [answers]
        except Exception as e:
            print(f"[ERROR] QA batch failed: {e}")
            counters["errors"] += len(qa_inputs)
            continue

        for (idx, para, sent, sentences), question, result in zip(batch_indices, questions, answers):
            question_text = question["generated_text"]
            if question_text.lower().startswith("what is the highlight"):
                continue
            answer, score = result["answer"].strip(), result["score"]
            if score < 0.15 or len(answer) < 4 or not any(c.isalnum() for c in answer):
                counters["skipped_filters"] += 1
                continue
            if answer.lower() in ["yes", "no", "maybe"]:
                counters["skipped_filters"] += 1
                continue
            if not any(answer.lower() in s.lower() for s in sentences):
                counters["skipped_filters"] += 1
                continue

            qa_pairs.append({
                "instruction": "Extract the correct answer span from the telecom document context.",
                "input": f"### Task: extractive_qa\n### Context:\n{para}\n\n### Question:\n{question_text}\n\n### Answer:",
                "output": answer,
                "source_doc": source,
                "chunk_id": idx,
                "confidence": round(score, 3)
            })
            counters["qa_generated"] += 1

    print(f"\U0001F50D Total Paragraphs: {counters['total_paragraphs']}")
    print(f" QA Pairs: {counters['qa_generated']}")
    print(f" Skipped: {counters['skipped_filters']} |  Errors: {counters['errors']}")
    return qa_pairs

# DOCX Processing
def process_docx_file(input_docx_path, output_jsonl_path=None, strategy="length"):
    """
    Load DOCX file, clean text, expand abbreviations using precomputed map,
    chunk with overlap, and return QA pairs (no writing to file).
    """
    input_path = Path(input_docx_path)
    doc = Document(input_path)

    # Combine raw text
    raw_text = "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

    # Step 1: Clean raw technical text
    cleaned = clean_technical_text(raw_text)

    # Step 2: Load abbreviation map
    try:
        with open("abbreviation_master_map.json", "r", encoding="utf-8") as f:
            abbreviation_map = json.load(f)
    except FileNotFoundError:
        abbreviation_map = {}

    # Step 3: Expand abbreviations in the cleaned text
    if abbreviation_map:
        pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in abbreviation_map.keys()) + r')\b')
        cleaned = pattern.sub(lambda m: f"{abbreviation_map[m.group()]} ({m.group()})", cleaned)

    # Step 4: Chunk with overlap
    paragraphs = split_with_overlap(cleaned, max_words=150, overlap=50)

    # Step 5: Generate QA pairs
    qa_pairs = generate_qa_pairs_from_paragraphs(paragraphs, source=input_path.name, strategy=strategy)

    # No saving to file — just return
    print(f" Generated {len(qa_pairs)} QA pairs from {input_path.name}")
    return qa_pairs


# Filtering Good QA Pairs
def filter_good_qa_pairs(input_jsonl_path, output_jsonl_path):
    qa_pairs = []
    with open(input_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            qa_pairs.append(json.loads(line))

    qa_df = pd.DataFrame(qa_pairs)
    qa_df["question"] = qa_df["input"].str.extract(r"### Question:\n(.+?)\n\n### Answer:")
    qa_df["context"] = qa_df["input"].str.extract(r"### Context:\n(.+?)\n\n### Question:", flags=re.DOTALL)

    qa_df["low_confidence_flag"] = qa_df["confidence"].astype(float) < 0.5
    qa_df["mismatch_flag"] = ~qa_df.apply(lambda row: row["output"].lower() in row["context"].lower(), axis=1)
    qa_df["vague_question_flag"] = qa_df["question"].str.lower().str.startswith("what is the highlight")

    qa_df["quality"] = "good"
    qa_df.loc[qa_df["low_confidence_flag"] | qa_df["mismatch_flag"] | qa_df["vague_question_flag"], "quality"] = "poor"

    good_qa_df = qa_df[qa_df["quality"] == "good"]
    with open(output_jsonl_path, "w", encoding="utf-8") as fout:
        for record in good_qa_df.to_dict(orient="records"):
            json.dump(record, fout, ensure_ascii=False)
            fout.write("\n")

    print(f" Saved {len(good_qa_df)} good QA pairs to {output_jsonl_path}")

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0


In [2]:
from pathlib import Path
import pandas as pd
import json

# Paths
base_path = Path("/mnt/data/TSpec-LLM/3GPP-clean")
final_output = Path("/mnt/data/First_RUN_final_3gpp_qa_filtered.jsonl")
stats_output = Path("/mnt/data/First_RUN_qa_stats_per_file.csv")
checkpoint_log = Path("/mnt/data/processed_files.txt")

# Load checkpoint log
if checkpoint_log.exists():
    processed_files = set(checkpoint_log.read_text(encoding="utf-8").splitlines())
else:
    processed_files = set()

# Reset final output (optional: only if you want to start fresh)
# final_output.unlink(missing_ok=True)
#docx_files = sorted(base_path.rglob("*.docx"))  # [:20] for testing

# Restrict to Rel-15 to Rel-19 only
rel_folders = [f"Rel-{i}" for i in range(15, 20)]  # 15, 16, 17, 18, 19
docx_files = []

for rel in rel_folders:
    rel_path = base_path / rel
    if rel_path.exists():
        docx_files.extend(sorted(rel_path.rglob("*.docx")))
    else:
        print(f" Skipping missing directory: {rel_path}")

stats = []

for docx_path in docx_files:
    file_id = docx_path.stem.replace(" ", "_")

    if file_id in processed_files:
        print(f" Skipping {file_id} (already processed)")
        continue

    try:
        # Run full pipeline → QA pairs in memory
        qa_pairs = process_docx_file(docx_path, output_jsonl_path=None, strategy="tfidf")
        good_qa_pairs = [pair for pair in qa_pairs if (
            pair["confidence"] >= 0.4 and
            pair["output"].lower() in pair["input"].lower() and
            not pair["input"].lower().startswith("what is the highlight")
        )]

        # Save directly to final file
        with open(final_output, "a", encoding="utf-8") as fout:
            for record in good_qa_pairs:
                json.dump(record, fout, ensure_ascii=False)
                fout.write("\n")

        stats.append({"file": docx_path.name, "qa_count": len(good_qa_pairs), "status": "success"})

        # Save file to checkpoint log
        with checkpoint_log.open("a", encoding="utf-8") as f:
            f.write(file_id + "\n")

    except Exception as e:
        stats.append({"file": docx_path.name, "qa_count": 0, "status": f"error: {str(e)}"})

# Save stats
stats_df = pd.DataFrame(stats)
stats_df.to_csv(stats_output, index=False)
stats_df.head()

⏩ Skipping 21101-f10 (already processed)
⏩ Skipping 21111-f20 (already processed)
⏩ Skipping 21201-f10 (already processed)
⏩ Skipping 21202-f00 (already processed)
⏩ Skipping 21205-f10 (already processed)
⏩ Skipping 21801-f20 (already processed)
⏩ Skipping 21866-f00 (already processed)
⏩ Skipping 21900-f20 (already processed)
⏩ Skipping 21905-f10 (already processed)
⏩ Skipping 21915-f00 (already processed)
⏩ Skipping 22001-f00 (already processed)
⏩ Skipping 22002-f00 (already processed)
⏩ Skipping 22003-f00 (already processed)
⏩ Skipping 22004-f00 (already processed)
⏩ Skipping 22011-f50 (already processed)
⏩ Skipping 22016-f00 (already processed)
⏩ Skipping 22022-f00 (already processed)
⏩ Skipping 22024-f00 (already processed)
⏩ Skipping 22030-f00 (already processed)
⏩ Skipping 22031-f00 (already processed)
⏩ Skipping 22032-f00 (already processed)
⏩ Skipping 22034-f00 (already processed)
⏩ Skipping 22038-f00 (already processed)
⏩ Skipping 22041-f00 (already processed)
⏩ Skipping 22042

Generating QA pairs: 100%|████████████████████████| 2/2 [00:04<00:00,  2.33s/it]


🔍 Total Paragraphs: 70
✅ QA Pairs: 23
🚫 Skipped: 46 | ❌ Errors: 0
✅ Generated 23 QA pairs from 33511-i10.docx


Generating QA pairs: 100%|████████████████████████| 2/2 [00:04<00:00,  2.12s/it]


🔍 Total Paragraphs: 92
✅ QA Pairs: 37
🚫 Skipped: 54 | ❌ Errors: 0
✅ Generated 37 QA pairs from 33512-i10.docx


...

Generating QA pairs: 100%|████████████████████████| 2/2 [00:04<00:00,  2.01s/it]


🔍 Total Paragraphs: 73
✅ QA Pairs: 29
🚫 Skipped: 44 | ❌ Errors: 0
✅ Generated 29 QA pairs from 38908-j00.docx


Generating QA pairs: 100%|████████████████████████| 3/3 [00:07<00:00,  2.62s/it]

🔍 Total Paragraphs: 166
✅ QA Pairs: 62
🚫 Skipped: 104 | ❌ Errors: 0
✅ Generated 62 QA pairs from 38922-j00.docx


,file,qa_count,status
0,23792-120_rm.docx,0,error: Opening and ending tag mismatch: rPr li...
1,RS+LDPC.docx,0,error: Opening and ending tag mismatch: rPr li...
2,Results for information from ASR vendors.docx,0,error: Opening and ending tag mismatch: rPr li...
3,TR 38.810 v2.4.0 - track changes.docx,0,error: Opening and ending tag mismatch: rPr li...
4,RS+LDPC.docx,0,error: Opening and ending tag mismatch: rPr li...
